## Run all notebooks

In [4]:
import nbformat
from nbconvert import HTMLExporter
from nbclient import NotebookClient
from pathlib import Path
import papermill as pm


In [5]:
def execute_notebook_with_papermill(notebook_path, outlier_rem_value, output_html, date, files_date, derivative,min_n_trials_per_condition):
    """Execute a Jupyter notebook with a specific `outlier_rem` value using Papermill.

    Args:
        notebook_path (str): Path to the notebook to execute.
        outlier_rem_value (bool): Value for `outlier_rem`.
        output_html (str): Path to save the HTML output of the notebook.
    """
    # Load the current notebook to get the kernel information
    with open(notebook_path, "r") as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Extract the kernel name used in the current notebook
    kernel_name = notebook.metadata.get('kernelspec', {}).get('name', 'python3')

    # Execute the notebook with Papermill
    executed_notebook_path = output_html.replace(".html", "_executed.ipynb")
    
    # Execute with parameters
    pm.execute_notebook(
        notebook_path,
        executed_notebook_path,
        parameters=dict(outlier_rem=outlier_rem_value, date=date, derivative=derivative, files_date=files_date), 
        kernel_name=kernel_name, min_n_trials_per_condition=min_n_trials_per_condition,
                start_node=0,      # Start execution from the first cell
        end_node=180    # Stop execution halfway/
        
    )

    # Convert the executed notebook to HTML
    from nbconvert import HTMLExporter
    html_exporter = HTMLExporter()
    html_exporter.exclude_input = True  # Exclude input cells if desired
    
    with open(executed_notebook_path, "r") as f:
        executed_notebook = nbformat.read(f, as_version=4)
        html_content, _ = html_exporter.from_notebook_node(executed_notebook)
    
    # Save HTML output
    with open(output_html, "w") as html_file:
        html_file.write(html_content)


### 1. Gap ET extraction and demographics

In [8]:
# Define paths
notebook_to_run = "C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/code/et/1_gap_et_extraction.ipynb"
out_dir = Path("papermill_outputs/gap_et_extraction")
out_dir.mkdir(exist_ok=True)

# Date
date = "2025_06_29"
files_date = None
derivative = None
outlier_rem = None
min_n_trials_per_condition= None
execute_notebook_with_papermill(
        notebook_path=notebook_to_run,
        outlier_rem_value=outlier_rem, date=date, files_date=files_date, derivative=derivative,
        min_n_trials_per_condition=min_n_trials_per_condition,
        output_html=str(out_dir / f"gap_et_extration_papermill.html")
    )


Passed unknown parameter: outlier_rem
Passed unknown parameter: derivative
Passed unknown parameter: files_date


Executing:   0%|          | 0/60 [00:00<?, ?cell/s]

In [9]:
### 2. Gap data preprocessing

# Define paths
notebook_to_run = "C:/Users/gabot/OneDrive - McGill University/Desktop/github_repos/q1k_neurosubs/code/et/2_gap_et_preprocessing.ipynb"
out_dir = Path("papermill_outputs/gap_et_preprocessing")
out_dir.mkdir(exist_ok=True)

# Date
date = "2025_06_29"
files_date = None
derivative = None
outlier_rem = None
min_n_trials_per_condition=6

execute_notebook_with_papermill(
        notebook_path=notebook_to_run,
        outlier_rem_value=outlier_rem, date=date, files_date=files_date, derivative=derivative,
        min_n_trials_per_condition=min_n_trials_per_condition,
        output_html=str(out_dir / f"gap_et_preprocessing_papermill.html")
    )

Passed unknown parameter: outlier_rem
Passed unknown parameter: date
Passed unknown parameter: derivative
Passed unknown parameter: files_date
Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/42 [00:00<?, ?cell/s]

In [3]:
# Run the Gap overlap notebooks wih each outliers

# Date
date = "01_24_2025"
files_date = "01_20_2025"
derivative = "standard" # "standard" or "no_pilot"
outlier_rem = False
execute_notebook_with_papermill(
        notebook_path=notebook_to_run,
        outlier_rem_value=outlier_rem, date=date, files_date=files_date, derivative=derivative,
        output_html=str(out_dir / f"output_outliers_removed_{date}.html")
    )


NameError: name 'out_dir' is not defined